In [3]:
# Importing liberaries
!pip install pandas
!pip install numpy
!pip install nltk
!pip install scikit-surprise
!pip install scikit-learn
!pip install collection
!pip install plotly
!pip install wordcloud
!pip install seaborn
!pip install networkx
import pandas as pd 
import numpy as np
from ast import literal_eval  # evaluate strings containing Python code in the current Python environment
from nltk.stem.snowball import SnowballStemmer # Removing stem words
from sklearn.feature_extraction.text import CountVectorizer  # To convert text to numerical data
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from collections import defaultdict
import matplotlib.pyplot as plt
import plotly.express as px
from wordcloud import WordCloud
import seaborn as sns
import networkx as nx

import warnings  # disable python warnings
warnings.filterwarnings("ignore")

You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
# Loading datasets 
!pip install ipykernel
movies_data = pd.read_csv("movies_metadata.csv", low_memory=False)
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')
links_small = pd.read_csv('links_small.csv')
ratings = pd.read_csv("ratings_small.csv")

You should consider upgrading via the 'C:\Users\shive\Documents\GitHub\Algo\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [7]:
# drop the timestamp column since we dont need it now
ratings = ratings.drop(columns="timestamp")

In [8]:
# fig = px.histogram(ratings, x="rating")
# fig.show()

In [9]:
# create reader from surprise 
# the rating should lie in the provided scale
reader = Reader(rating_scale =(0.5,5))

In [10]:
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

In [11]:
print(data)

In [12]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

In [13]:
# create trainset ie the data which is present (ratings of those movies which are rated by respective users)
trainset = data.build_full_trainset()

# create testset, here the anti_testset is testset
# data containing users movie pairs which are not rated by that particular user
testset = trainset.build_anti_testset()

In [14]:
sim_options = {
    "name": ["msd", "cosine",'pearson_baseline'],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}
param_grid = {"sim_options": sim_options}


gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)

In [15]:
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [16]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

0.9287261868332903
{'sim_options': {'name': 'msd', 'min_support': 3, 'user_based': True}}


In [17]:
params=gs.best_params['rmse']
model=KNNWithMeans(sim_options=params['sim_options'],n_epochs=10)

In [18]:
cross_validate(model, data, measures=['RMSE','MAE'], cv= 5)
print('Training Done')

#prediction
prediction = model.test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Training Done


In [19]:
example = { 'userId' : [99999,99999,99999,99999,99999],
           'movieId' : [31,1029,1293,1172,1343],
           'rating'  : [3.0, 4.5, 1.2, 3.3,2]
    
}

df = pd.DataFrame(example)
frames = [ratings, df]
result = pd.concat(frames)

In [20]:
# Removing the rows with null value in the vote_average and vote_count columns in movies_data dataframe

movies_data = movies_data.dropna(subset=['vote_average', 'vote_count'])
#Preprocessing the data

movies_data['id'] = movies_data['id'].astype('int')
# merging both credits and keywords in movies_data on the basis of movie id
movies_data = movies_data.merge(credits, on='id')
movies_data = movies_data.merge(keywords, on='id')

In [21]:
columns = ['userId','movieId','rating']
#create dataset from dataframe
data= Dataset.load_from_df(result[columns],reader)

#create trainset
trainset= data.build_full_trainset()

#create testset, here the anti_testset is testset
testset = trainset.build_anti_testset()

cross_validate(model,data, measures=['RMSE','MAE'], cv= 5) #cv is the number of parts in which data will be divided.
print('Training Done')

#prediction
prediction = model.test(testset)
prediction[99999]

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Training Done


Prediction(uid=12, iid=4344, r_ui=3.5435710786029255, est=2.242729281641224, details={'actual_k': 24, 'was_impossible': False})

In [22]:
def get_top_n(prediction, n):
    
    '''
    This function recommend users with top n movies based on prediction calculated using the surprise library
    
    Parameters: prediction(list): This contains (user, movie) rating prediction for all user movie pairs
                n(int): Number of recommendations
    
    Results: Returns top 30 movies along with movie id for all users
    
    
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in prediction: 
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the n highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

rcmnd = []
top_n = get_top_n(prediction, n=30)
for uid, user_ratings in top_n.items():
    if uid == 99999:
        for (iid,rating) in user_ratings:
            for i in range(movies_data.shape[0]):
                if movies_data['id'][i] == iid:
                    rcmnd.append([movies_data['id'][i],movies_data['title'][i]])
        break

In [23]:
rcmnd[:10]

[[1371, 'Rocky III'],
 [1405, 'Greed'],
 [2105, 'American Pie'],
 [2193, 'My Tutor'],
 [2294, 'Jay and Silent Bob Strike Back'],
 [2455, 'Confidentially Yours'],
 [17, 'The Dark'],
 [62, '2001: A Space Odyssey'],
 [110, 'Three Colors: Red'],
 [144, 'Wings of Desire']]

In [24]:
# Tinkter

In [25]:
import tkinter as tk
import pickle
import bz2
import _pickle as cPickle
with open('movie_list.pkl', 'rb') as f:
    df = pickle.load(f)

with open('final_prediction_list.pkl', 'rb') as f:
    final_prediction_list = pickle.load(f)
l2 = [None for i in range(10)]
def recommend(movie,n=5):
    
    for i in range(10):
        if l2[i] is not None:
            l2[i].grid_remove()
    try:
        index = df[df['original_title'] == movie.lower()].index[0]
        for i in range(n):
            moive_index=final_prediction_list[index][i]
            movie_name=df['original_title'][moive_index]
            print(df['original_title'][moive_index])
            l2[i] = Label(root ,  text = movie_name)
            l2[i].grid(row = 2+i, column = 6, sticky = W, pady = 5)       
    except Exception as e:
        print(e)
        print("Movie Not in Database")

In [26]:
import tkinter as tk
import tkinter.ttk
from tkinter import *
import tkinter.messagebox

In [27]:

l = [None for i in range(10)]

def genre_based():
    '''
       Callback function used for the submit button on the interface

       This function takes input directly from the Tkinter interface and based on the type of input provided it finds recommendation for the user and provide output on the Tkinter interface.

       Parameters: None

       Returns: None 

    ''' 

    for i in range(10):
        if l[i] is not None:
            l[i].grid_remove()
    event = "Select"

    # produced recommendations for the user based on the rating provided to selected movies if no genre is selected else recommend top movies from the selected genres
    if event == "Select":
        rtts = list()
        rtts.append(l11.get())
        rtts.append(l12.get())
        rtts.append(l13.get())
        rtts.append(l14.get())
        rtts.append(l15.get())
        movie_ratings = [int(i) for i in rtts]
        print(movie_ratings)
        
        example = { 'userId' : [99999,99999,99999,99999,99999],
           'movieId' : [278, 13,637,122, 11],
           'rating'  : movie_ratings,
                  }
        
        df = pd.DataFrame(example)
        frames = [ratings, df]
        result = pd.concat(frames)
        
        #create dataset from dataframe
        data= Dataset.load_from_df(result[columns],reader)

        #create trainset
        trainset= data.build_full_trainset()

        #create testset, here the anti_testset is testset
        testset = trainset.build_anti_testset()

        cross_validate(model,data, measures=['RMSE','MAE'], cv= 5, verbose= True)
        print('Training Done')

        #prediction
        prediction = model.test(testset)
        prediction[99999]

        from collections import defaultdict
        def get_top_n(prediction, n):

            # First map the predictions to each user.
            top_n = defaultdict(list)
            for uid, iid, true_r, est, _ in prediction:
                top_n[uid].append((iid, est))

            # Then sort the predictions for each user and retrieve the n highest ones.
            for uid, user_ratings in top_n.items():
                user_ratings.sort(key=lambda x: x[1], reverse=True)
                top_n[uid] = user_ratings[:n]

            return top_n

        rcmnd = []
        top_n = get_top_n(prediction, n=30)
        for uid, user_ratings in top_n.items():
            if uid == 99999:
                for (iid,rating) in user_ratings:
                    for i in range(movies_data.shape[0]):
                        if movies_data['id'][i] == iid:
                            rcmnd.append([movies_data['id'][i],movies_data['title'][i]])
                break
    
        for i in range(min(10,len(rcmnd))):
            l[i] = Label(root ,  text = rcmnd[i][1])
            l[i].grid(row = 2+i, column = 5, sticky = W, pady = 5)                

In [28]:
# temp dataframe contains movies which are present in both ratings and movies_data dataframe

id_list = list(ratings['movieId'])
temp = movies_data.loc[movies_data['id'].isin(id_list)]
temp = temp.reset_index()
print(temp[['id' , 'title']])
temp['title'][1]

         id                       title
0       949                        Heat
1       710                   GoldenEye
2      1408            Cutthroat Island
3       524                      Casino
4      4584       Sense and Sensibility
...     ...                         ...
2843  80831                Sang Pemimpi
2844   3104  Frankenstein Created Woman
2845  64197        Travelling with Pets
2846  98604                  Cinderella
2847  49280            The One-Man Band

[2848 rows x 2 columns]


'GoldenEye'

In [29]:


# create root window
root = Tk()

# root window title and dimension4
root.title("AZ Movie Recommender System")

# Set geometry (widthxheight)
root.geometry('700x400')

''

In [30]:

# adding a label to the root window
l1 = Label(root, text = "MOVIE RECOMMENDER SYSTEM", fg = "blue")
l1.grid(row = 0,column = 3)

# datatype of menu text
clicked = StringVar()

# initial menu text
clicked.set( "Select Genre" )


l3 = Label(root, text = "Rate the following movies")
l3.grid(row = 1,column = 2,sticky = W, pady = 2)

# labels for movies name
l4 = Label(root, text="Movies Name").grid(row=2, column=2)
l5 = Label(root, text="the shawshank redemption").grid(row=3, column=2) # 278
l6 = Label(root, text="Cutthroat Island").grid(row=4, column=2) # 13
l7 = Label(root, text="life is beautiful").grid(row=5, column=2) #637
l8 = Label(root, text="The One-Man Band").grid(row=6, column=2) #122
l9 = Label(root, text="star wars").grid(row=7, column=2) # 11

# label for movies rating
l10 = Label(root, text="Rate the movie on the scale of 5").grid(row=2, column=3)
l11 = Spinbox(root, from_= 0, to = 5)
l11.grid(row=3, column=3)
l12 = Spinbox(root, from_= 0, to = 5)
l12.grid(row=4, column=3)
l13= Spinbox(root, from_= 0, to = 5)
l13.grid(row=5, column=3)
l14 = Spinbox(root, from_= 0, to = 5)
l14.grid(row=6, column=3)
l15 = Spinbox(root, from_= 0, to = 5)
l15.grid(row=7, column=3)


# button widget with green color text
button = Button(root, text = "SUBMIT" , fg = "white",bg = "green", command = genre_based)
button.grid(row = 10,column = 3, sticky = S)

# Creating seperators for better UI 
x2 = tkinter.ttk.Separator(root, orient=VERTICAL).grid(column=4, row=1, rowspan=12, sticky='ns')

l16 = Label(root, text="Results").grid(row=1, column= 5, sticky = W, pady=10)




In [31]:
# start the program
root.mainloop()

[3, 3, 4, 4, 4]
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9208  0.9206  0.9144  0.9179  0.9214  0.9190  0.0026  
MAE (testset)     0.7047  0.7036  0.7005  0.7000  0.7064  0.7030  0.0024  
Fit time          0.17    0.17    0.25    0.27    0.25    0.22    0.04    
Test time         1.08    1.07    1.37    1.27    1.27    1.21    0.12    
Training Done
